In [1]:
# Step 1.필요한 라이브러리 로딩
import urllib.request , urllib
import pandas as pd    
from bs4 import BeautifulSoup
import os, tempfile, time, math, random
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from urllib.parse import urlparse
import re

#Step 2. 사용자에게 검색어 키워드를 입력 받습니다.
print("=" *80)
print(" 올리브영 Best상품 이미지 다운로드 ")
print("=" *80)
cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본경로:c:\\py_temp\\):")
if f_dir == '' :
    f_dir = "c:\\py_temp\\"
    
print("\n")
if cnt > 60 :
      print("요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~")
else :
      print("요청하신 데이터를 수집하고 있으니 잠시만 기다려 주세요~~")

#Step 3.저장될 폴더 생성
query_txt='올리브영'
n = time.localtime()
s1 = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
base_dir = f_dir + s1 + '-' + query_txt + '-이미지'
os.makedirs(base_dir, exist_ok=True)

s_time = time.time()

# 사람처럼 행동하는 함수들
def human_sleep(min_sec=1, max_sec=3):
    """랜덤 대기로 봇 감지 회피"""
    time.sleep(random.uniform(min_sec, max_sec))

def human_scroll(driver, scroll_pause_min=1.5, scroll_pause_max=3.5):
    """사람처럼 불규칙하게 스크롤"""
    scroll_amount = random.randint(800, 1500)
    driver.execute_script(f"window.scrollBy(0, {scroll_amount});")
    human_sleep(scroll_pause_min, scroll_pause_max)

# 파일명에 사용 불가능한 문자 제거
def sanitize_filename(filename):
    """파일명에서 특수문자 제거"""
    # Windows에서 사용 불가능한 문자 제거
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    # 공백을 언더스코어로
    filename = filename.replace(' ', '_')
    # 너무 길면 자르기 (최대 50자)
    if len(filename) > 50:
        filename = filename[:50]
    return filename

# ★★★ 상품설명 탭에서 이미지 다운로드 ★★★
def download_product_images(driver, product_url, folder_path, main_url):
    """
    상품 상세페이지에서:
    1. 상품설명 탭 클릭
    2. 이미지 URL 수집
    3. 이미지 다운로드
    """
    try:
        print(f"     → 상세페이지 접속 중...")
        driver.get(product_url)
        human_sleep(3, 4)
        
        # 페이지 로딩 대기
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.TAG_NAME, "body"))
            )
        except:
            pass
        
        # ===== 1. "상품설명 더보기" 버튼 클릭 =====
        print(f"     → '상품설명 더보기' 버튼 클릭 시도...")
        try:
            # 스크롤 조금 내리기 (버튼이 보이도록)
            driver.execute_script("window.scrollBy(0, 400);")
            time.sleep(2)
            
            all_buttons = driver.find_elements(By.TAG_NAME, "button")
            button_clicked = False
            
            for btn in all_buttons:
                try:
                    btn_text = btn.text.strip()
                    # "상품설명 더보기" 또는 유사한 텍스트
                    if "상품설명" in btn_text and "더보기" in btn_text:
                        driver.execute_script("arguments[0].click();", btn)
                        print(f"     ✓ '{btn_text}' 버튼 클릭 완료!")
                        button_clicked = True
                        time.sleep(3)  # 콘텐츠 로딩 대기
                        break
                except:
                    continue
            
            if not button_clicked:
                print(f"     ℹ️ '상품설명 더보기' 버튼 없음, 현재 페이지에서 이미지 수집")
                
        except Exception as e:
            print(f"     ⚠️ 버튼 클릭 실패: {str(e)}")
        
        # 스크롤 (이미지 로딩)
        # 스크롤 (이미지 로딩)  ★ 여기 전체를 교체
        print(f"     → 상세페이지 전체를 끝까지 스크롤하며 이미지 로딩 중...")

        max_loops = 30          # 최대 몇 번까지 시도할지 (안전장치)
        same_count = 0          # 페이지 높이가 안 바뀐 횟수

        # 현재 페이지 전체 높이 읽기
        last_height = driver.execute_script("return document.body.scrollHeight")

        for _ in range(max_loops):
            # 한 번에 화면 조금 넘게 내리기
            driver.execute_script("window.scrollBy(0, 1200);")
            time.sleep(1.0)  # 이미지 로딩 기다리기

            # 스크롤 후 페이지 높이 다시 읽기
            new_height = driver.execute_script("return document.body.scrollHeight")

            if new_height == last_height:
                # 높이가 그대로면 same_count 증가
                same_count += 1

                # 2번 연속 높이가 안 바뀌면 → 더 이상 로딩되는 내용 없음 → 종료
                if same_count >= 2:
                    break
            else:
                # 높이가 바뀌면 → 아직 로딩 중이란 뜻 → 카운트 초기화
                same_count = 0
                last_height = new_height

        # 맨 밑까지 갔다가 살짝 위로 올려주기(선택)
        driver.execute_script("window.scrollBy(0, -400);")
        time.sleep(1.0)

        
        # ===== 2. 이미지 URL 수집 =====
        print(f"     → 이미지 URL 수집 중...")
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        
        # ★ div.speedycat-container 안의 이미지만!
        image_urls = []
        
        speedycat_containers = soup.select('div.speedycat-container')
        
        if not speedycat_containers:
            print(f"     ⚠️ speedycat-container를 찾을 수 없음")
        else:
            print(f"     ✓ {len(speedycat_containers)}개 speedycat-container 발견!")
            
            for container in speedycat_containers:
                imgs = container.find_all('img')
                for img in imgs:
                    src = img.get('src')
                    if src:
                        # 상대 경로면 절대 경로로 변환
                        if src.startswith('//'):
                            src = 'https:' + src
                        elif src.startswith('/'):
                            src = 'https://www.oliveyoung.co.kr' + src
                        
                        # ★ GIF 제외! JPG, PNG만!
                        if src.startswith('http'):
                            # GIF 파일 제외
                            if src.lower().endswith('.gif'):
                                continue
                            if src not in image_urls:
                                image_urls.append(src)
        
        print(f"     ✓ 총 {len(image_urls)}개 이미지 URL 발견! (GIF 제외)")
        
        if not image_urls:
            print(f"     ⚠️ 이미지를 찾을 수 없음")
            driver.get(main_url)
            human_sleep(2, 3)
            return 0
        
        # ===== 3. 이미지 다운로드 =====
        print(f"     → 이미지 다운로드 중...")
        
        downloaded_count = 0
        folder_num = count + 1  # 현재 폴더 번호
        
        for idx, img_url in enumerate(image_urls, 1):
            try:
                # URL에서 쿼리 파라미터 제거
                clean_url = img_url.split('?')[0]
                
                # 파일 확장자
                ext = os.path.splitext(clean_url)[1].lower()
                if not ext or ext not in ['.jpg', '.jpeg', '.png']:
                    ext = '.jpg'
                
                # ★ 파일명: 01_1.jpg, 01_2.jpg, 01_3.jpg...
                filename = f"{folder_num:02d}_{idx}{ext}"
                filepath = os.path.join(folder_path, filename)
                
                # 다운로드
                response = requests.get(img_url, timeout=10, headers={
                    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
                })
                
                if response.status_code == 200:
                    with open(filepath, 'wb') as f:
                        f.write(response.content)
                    downloaded_count += 1
                    print(f"       [{downloaded_count}/{len(image_urls)}] {filename} 저장 완료")
                
            except Exception as e:
                print(f"       ⚠️ 이미지 {idx} 다운로드 실패: {str(e)}")
                continue
        
        print(f"     ✅ {downloaded_count}개 이미지 다운로드 완료!")
        
        # 목록으로 복귀
        print(f"     ← 목록 페이지로 복귀")
        driver.get(main_url)
        human_sleep(2, 3)
        
        return downloaded_count
        
    except Exception as e:
        print(f"     ❌ 오류: {str(e)}")
        try:
            driver.get(main_url)
            human_sleep(2, 3)
        except:
            pass
        return 0

# Step 4. Chrome 드라이버 설정
print("\n🔧 Chrome 드라이버 설정 중...")

CHROME_PATH = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
VERSION_MAIN = 142

options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--start-maximized')

user_data = os.path.join(tempfile.gettempdir(), f"uc_profile_{int(time.time())}")
os.makedirs(user_data, exist_ok=True)

print("🚀 Chrome 브라우저 시작 중...")

try:
    driver = uc.Chrome(
        options=options,
        browser_executable_path=CHROME_PATH,
        version_main=VERSION_MAIN,
        user_data_dir=user_data,
        use_subprocess=False,
        headless=False,
    )
    print("✅ Chrome 드라이버 초기화 완료!")
except Exception as e:
    print(f"❌ 드라이버 초기화 실패: {str(e)}")
    driver = uc.Chrome(
        browser_executable_path=CHROME_PATH,
        version_main=VERSION_MAIN,
    )

# Step 5-7. 페이지 이동
print("\n📡 브라우저 워밍업 중...")
try:
    driver.get("https://www.naver.com")
    human_sleep(2, 3)
    print("✅ 워밍업 완료!")
except:
    pass

print("\n📡 올리브영 메인 페이지 접속 중...")
try:
    driver.get("https://www.oliveyoung.co.kr")
    human_sleep(3, 5)
    print("✅ 메인 페이지 접속 완료!")
    
    for i in range(2):
        driver.execute_script(f"window.scrollBy(0, {random.randint(300, 600)});")
        human_sleep(1, 2)
except:
    pass

print("\n📡 베스트 100 페이지로 이동 중...")
target_url = "https://www.oliveyoung.co.kr/store/main/getBestList.do?dispCatNo=900000100100001&fltDispCatNo=&pageIdx=1&rowsPerPage=8&t_page=%EB%9E%AD%ED%82%B9&t_click=%ED%8C%90%EB%A7%A4%EB%9E%AD%ED%82%B9_%EC%A0%84%EC%B2%B4"

try:
    driver.get(target_url)
    human_sleep(4, 6)
    if "oliveyoung" in driver.current_url:
        print("✅ 올리브영 페이지 접속 성공!")
    else:
        print("수동으로 이동 후 Enter...")
        input()
except:
    print("수동으로 이동 후 Enter...")
    input()

print("\n===== 곧 수집된 결과를 출력합니다 ^^ =====\n")

# Step 8-9. 페이지 로딩 및 스크롤
print("⏳ 페이지 로딩 대기 중...")
human_sleep(3, 5)

try:
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "ul.cate_prd_list"))
    )
    print("✅ 상품 리스트 로드 완료!")
except:
    print("상품이 보이면 Enter...")
    input()

print("\n📜 스크롤하여 상품 로딩 중...")
last_height = driver.execute_script("return document.body.scrollHeight")
scroll_count = 0
no_change_count = 0

while scroll_count < 25:
    human_scroll(driver, 1.5, 3.5)
    
    if random.random() < 0.15:
        scroll_back = random.randint(-300, -100)
        driver.execute_script(f"window.scrollBy(0, {scroll_back});")
        human_sleep(0.5, 1.5)
    
    new_height = driver.execute_script("return document.body.scrollHeight")
    scroll_count += 1
    
    if scroll_count % 3 == 0:
        print(f"  스크롤 {scroll_count}회 완료...")
    
    if new_height == last_height:
        no_change_count += 1
        if no_change_count >= 4:
            print("  📌 더 이상 로드할 상품이 없습니다.")
            break
    else:
        no_change_count = 0
    
    last_height = new_height

print("  페이지 상단으로 이동 중...")
for i in range(5):
    scroll_up = -last_height // 5
    driver.execute_script(f"window.scrollBy(0, {scroll_up});")
    human_sleep(0.3, 0.8)

human_sleep(2, 3)

# Step 10. HTML 파싱
print("\n🔍 HTML 파싱 중...")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

product_list = soup.select("ul.cate_prd_list li")
if not product_list:
    product_list = soup.select("div.prd_list li")

print(f"\n✅ 총 {len(product_list)}개 상품 발견!")

if len(product_list) < 10:
    print("상품이 적습니다. 확인 후 Enter...")
    input()
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    product_list = soup.select("ul.cate_prd_list li")

print(f"📊 요청하신 {cnt}개 수집 시작...\n")
print("="*100)

# 수집
count = 0
total_images = 0

for idx, item in enumerate(product_list):
    if count >= cnt:
        break
    
    try:
        # 상품이름
        brand_elem = item.select_one("span.tx_brand")
        if not brand_elem:
            brand_elem = item.select_one("p.brand")
        brand = brand_elem.text.strip() if brand_elem else "브랜드없음"
        
        title_elem = item.select_one("p.tx_name")
        if not title_elem:
            title_elem = item.select_one("p.name")
        title = title_elem.text.strip() if title_elem else "상품명없음"
        1
        # ★ 폴더명: 01, 02, 03... (단순 숫자만)
        folder_name = f"{count+1:02d}"
        folder_path = os.path.join(base_dir, folder_name)
        os.makedirs(folder_path, exist_ok=True)
        
        # URL
        link_elem = item.select_one("a[data-ref-goodsno]")
        if not link_elem:
            link_elem = item.select_one("a")
        
        if link_elem and link_elem.get('href'):
            product_url = link_elem.get('href')
            if not product_url.startswith('http'):
                product_url = "https://www.oliveyoung.co.kr" + product_url
        else:
            product_url = ""
        
        print(f"\n[{count+1:3d}] {brand} {title[:50]}")
        print(f"     📁 폴더: {folder_name}")
        
        # 이미지 다운로드
        if product_url:
            img_count = download_product_images(driver, product_url, folder_path, target_url)
            total_images += img_count
        else:
            print(f"     ❌ URL 없음")
        
        count += 1
        
        if count % 5 == 0:
            print(f"\n{'='*100}")
            print(f"  ✅ {count}개 완료! (총 {total_images}개 이미지 다운로드)")
            print(f"{'='*100}")
            human_sleep(2, 3)
        
    except Exception as e:
        print(f"     ❌ 오류: {str(e)}")
        continue

print(f"\n{'='*100}")
print(f"✅ 총 {count}개 상품 처리 완료!")
print(f"📸 총 {total_images}개 이미지 다운로드!")
print(f"{'='*100}\n")

driver.quit()
print("🔒 브라우저 종료\n")

e_time = time.time()
t_time = e_time - s_time

print(f"\n{'='*100}")
print(f"🎉 완료!")
print(f"⏱️  소요 시간: {t_time:.2f}초 ({t_time/60:.1f}분)")
print(f"📊 처리 상품: {count}개")
print(f"📸 다운로드 이미지: {total_images}개")
print(f"📁 저장 위치: {base_dir}")
print(f"{'='*100}")

 올리브영 Best상품 이미지 다운로드 


1.크롤링 할 건수는 몇건입니까?:  100
2.파일을 저장할 폴더명만 쓰세요(기본경로:c:\py_temp\): 




요청 건수가 많아서 시간이 제법 소요되오니 잠시만 기다려 주세요~~

🔧 Chrome 드라이버 설정 중...
🚀 Chrome 브라우저 시작 중...
✅ Chrome 드라이버 초기화 완료!

📡 브라우저 워밍업 중...
✅ 워밍업 완료!

📡 올리브영 메인 페이지 접속 중...
✅ 메인 페이지 접속 완료!

📡 베스트 100 페이지로 이동 중...
✅ 올리브영 페이지 접속 성공!

===== 곧 수집된 결과를 출력합니다 ^^ =====

⏳ 페이지 로딩 대기 중...
✅ 상품 리스트 로드 완료!

📜 스크롤하여 상품 로딩 중...
  스크롤 3회 완료...
  📌 더 이상 로드할 상품이 없습니다.
  페이지 상단으로 이동 중...

🔍 HTML 파싱 중...

✅ 총 100개 상품 발견!
📊 요청하신 100개 수집 시작...


[  1] 메디힐 [2025 어워즈/14년연속 누적판매 1위] 메디힐 에센셜 마스크팩 10+2/10매 7종 
     📁 폴더: 01
     → 상세페이지 접속 중...
     → '상품설명 더보기' 버튼 클릭 시도...
     ✓ '상품설명 더보기' 버튼 클릭 완료!
     → 상세페이지 전체를 끝까지 스크롤하며 이미지 로딩 중...
     → 이미지 URL 수집 중...
     ✓ 55개 speedycat-container 발견!
     ✓ 총 42개 이미지 URL 발견! (GIF 제외)
     → 이미지 다운로드 중...
       [1/42] 01_1.png 저장 완료
       [2/42] 01_2.png 저장 완료
       [3/42] 01_3.png 저장 완료
       [4/42] 01_4.jpeg 저장 완료
       [5/42] 01_5.jpeg 저장 완료
       [6/42] 01_6.jpeg 저장 완료
       [7/42] 01_7.jpeg 저장 완료
       [8/42] 01_8.jpeg 저장 완료
       [9/42] 01_9.jpeg 저장 완료
       [10/42